In [64]:
import pandas as pd
import numpy as np
import os
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from time import time
from IPython.display import clear_output
from keras .preprocessing.text import one_hot, Tokenizer
from keras.utils import pad_sequences
import nltk
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, Input
from keras.models import Model
import keras
import json

In [65]:
dataDir = os.listdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data")
os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data")

In [66]:
def check(token):
    for i in range(len(token)):
        if ord(token[i]) not in range(97, 123):
            return False
    return True

In [67]:
corpus = []
for file in os.listdir("OCR"):
    df = pd.read_csv("OCR/" + file)
    text = df["text"].to_numpy()
    tokens = []
    for t in text:
        T = t.lower()
        while len(T) >0 and ord(T[0]) not in range(97, 123):
            T = T[1:]
        while len(T)>0 and ord(T[-1]) not in range(97, 123):
            T = T[:-1]
        if len(T)>1 and check(T):
            tokens.append(T)
    T = " ".join(tokens)
    corpus.append(T)



In [68]:
text = " ".join(corpus)

In [69]:
tokens = text.split(" ")
len(tokens)

18039

In [70]:
valid = []
for t in tokens:
    if t not in valid:
        valid.append(t)

In [71]:
len(valid)

3551

In [72]:
len(corpus)

110

In [73]:
vocab_size = 4000
encoded_corp = []
for i, doc in enumerate(corpus):
    encoded_corp.append(one_hot(doc, vocab_size))
    print("Encoding for doc ", i+1, " is : ", one_hot(doc, vocab_size))


Encoding for doc  1  is :  [926, 902, 3428, 3766, 1198, 917, 3147, 917, 3917, 2827, 926, 917, 838, 3353, 1848, 3419, 957, 1833, 2676, 2054, 808, 2524, 3090, 330, 3891, 2762, 1461, 3826, 1265, 1703, 696, 1990, 1045, 3980, 1990, 1045, 1632, 245, 1956, 2181, 6, 1814, 3282, 2916, 6, 1814, 3375]
Encoding for doc  2  is :  [164, 926, 745, 926, 1418, 2192, 465, 3915, 872, 1925, 3061, 1444, 1948, 2716, 3660, 3147, 1390, 406, 695, 3782, 3813, 985, 1951, 120, 1418, 917, 2687, 2274, 3908, 3061, 149, 3867, 1833, 3428, 1418, 2192, 2990, 2273, 3256, 295, 2628, 2801, 1252, 586, 497, 1461, 3282, 641, 1418, 3147, 1390, 917, 3826, 1265, 3531, 703, 761, 1283, 3782, 3813, 3646, 1951, 1944, 406, 3147, 3524, 2873, 406, 1010, 2284, 783, 2366, 1052, 1418, 2596, 3503, 3804, 1635, 3464, 1828, 3618, 1024, 3281, 1052, 1214, 3976, 22, 18, 47, 1244, 226, 1244, 2551, 684, 2419, 3331, 3320, 2552, 3909, 1030, 1722, 1188, 164, 3281, 2659, 2309, 783, 1052, 164, 1052, 3281, 164, 1052, 3976, 22, 18, 2552, 226, 2996, 2551,

In [74]:
maxlen = -1
for doc in corpus:
    tokens = nltk.word_tokenize(doc)
    if maxlen<len(tokens):
        maxlen = len(tokens)
maxlen

426

In [75]:
pad_corpus = pad_sequences(encoded_corp, maxlen=maxlen, padding="post", value=0)

In [76]:
OD = 1

In [77]:
input = Input(shape=(len(corpus), maxlen), dtype="float64")

In [78]:
word_input = Input(shape=(maxlen, ), dtype="float64")

word_embeddings = Embedding(input_dim=vocab_size, output_dim=OD, input_length=maxlen)(word_input)

word_vec = Flatten()(word_embeddings)

embed_model = Model([word_input], word_vec)

In [79]:
embed_model.compile(optimizer=keras.optimizers.Adam(), loss="binary_crossentropy", metrics=["acc"])

In [80]:
embed_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 426)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 426, 1)            4000      
                                                                 
 flatten_2 (Flatten)         (None, 426)               0         
                                                                 
Total params: 4,000
Trainable params: 4,000
Non-trainable params: 0
_________________________________________________________________


In [81]:
embeddings = embed_model.predict(pad_corpus)

1/4 [======>.......................] - ETA: 0s

4/4 [==============================] - 0s 1ms/step


In [82]:
embeddings.shape

(110, 426)

In [83]:
embeddings

array([[-0.01062278, -0.02553654,  0.01180551, ...,  0.0355869 ,
         0.0355869 ,  0.0355869 ],
       [ 0.01264283, -0.01062278,  0.0337472 , ...,  0.0355869 ,
         0.0355869 ,  0.0355869 ],
       [ 0.00302632, -0.01677337, -0.01677337, ...,  0.0355869 ,
         0.0355869 ,  0.0355869 ],
       ...,
       [ 0.0156909 , -0.0455268 , -0.01941619, ...,  0.0355869 ,
         0.0355869 ,  0.0355869 ],
       [-0.01062278,  0.03873867, -0.04369842, ...,  0.0355869 ,
         0.0355869 ,  0.0355869 ],
       [-0.01323012,  0.0337472 ,  0.01478812, ...,  0.0355869 ,
         0.0355869 ,  0.0355869 ]], dtype=float32)

In [84]:
embeddings = embeddings.reshape(-1, maxlen, OD)
embeddings

array([[[-0.01062278],
        [-0.02553654],
        [ 0.01180551],
        ...,
        [ 0.0355869 ],
        [ 0.0355869 ],
        [ 0.0355869 ]],

       [[ 0.01264283],
        [-0.01062278],
        [ 0.0337472 ],
        ...,
        [ 0.0355869 ],
        [ 0.0355869 ],
        [ 0.0355869 ]],

       [[ 0.00302632],
        [-0.01677337],
        [-0.01677337],
        ...,
        [ 0.0355869 ],
        [ 0.0355869 ],
        [ 0.0355869 ]],

       ...,

       [[ 0.0156909 ],
        [-0.0455268 ],
        [-0.01941619],
        ...,
        [ 0.0355869 ],
        [ 0.0355869 ],
        [ 0.0355869 ]],

       [[-0.01062278],
        [ 0.03873867],
        [-0.04369842],
        ...,
        [ 0.0355869 ],
        [ 0.0355869 ],
        [ 0.0355869 ]],

       [[-0.01323012],
        [ 0.0337472 ],
        [ 0.01478812],
        ...,
        [ 0.0355869 ],
        [ 0.0355869 ],
        [ 0.0355869 ]]], dtype=float32)

In [99]:
dict = {}
for i in range(len(corpus)):
    doc = corpus[i].split(" ")
    for j in range(len(doc)):
        dict[doc[j]] = embeddings[i][j]


In [100]:
from sklearn.metrics.pairwise import cosine_similarity

In [101]:
print(cosine_similarity([dict["date"]], [dict["dated"]]))

[[-1.]]


In [102]:
dict["invoice"][0]

-0.010622777

In [103]:
for key in dict.keys():
    val = dict[key][0]
    val = float(val)
    dict[key] = val
dict

{'invoice': -0.010622777044773102,
 'pitstop': -0.025536537170410156,
 'order': 0.011805512011051178,
 'id': 0.04954290762543678,
 'service': -0.0013811588287353516,
 'date': 0.0017180219292640686,
 'delivery': 0.0387386716902256,
 'registration': 0.023914668709039688,
 'number': 0.0473020114004612,
 'renault': -0.009304143488407135,
 'pulse': -0.04712529107928276,
 'diesel': 0.021498266607522964,
 'odometer': -0.04326442629098892,
 'from': -0.0322084054350853,
 'to': 0.04574406519532204,
 'cars': -0.013807021081447601,
 'care': -0.04614464193582535,
 'opp': -0.029809260740876198,
 'trident': -0.041257403790950775,
 'hyundai': -0.04387914016842842,
 'showroom': 0.039756570011377335,
 'kudlu': -0.003694199025630951,
 'gate': -0.009395528584718704,
 'hosur': -0.029435468837618828,
 'main': -0.009125340729951859,
 'road': -0.019274592399597168,
 'bengaluru': 0.03086050972342491,
 'gst': -0.038604985922575,
 'bumper': 0.025290433317422867,
 'bracket': 0.01819659397006035,
 'front': -0.0297

In [104]:
pwd

"/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Models"

In [105]:
os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Models")

In [106]:
with open("wordEmbeddings.json", "w") as file:
    json.dump(dict, file)

In [107]:
refDict = {
    "invoice" : ["bill", "receipt", "inv"],
    "date" : ["dated", "dt"],
}

In [108]:
vocab = ["invoice", "inv", "receipt", "bill", "order", "payment", "due", "date", "dated"]

In [ ]:
vocab = ["invoice", "inv", "receipt", "bill", "order", "payment", "due", "date", "dated"]
emb = {
    "invoice" : 1,
    "inv" : 1,
    "bill" : 1,
    "receipt" : 1,
    "order" : 2,
    "payment" : 3,
    "due" : 4,
    "date" : 5,
    "dated" : 5
}